LLM weights is usually not enough to give accurate and insightful answers to our questions thus we need external tools to access the better functionality and web/world.

In [22]:
import os
import re
from groq import Groq
from dotenv import load_dotenv
import requests
import json

In [25]:

def get_weather(location):
    """
    Fetches weather data for a given city using OpenWeatherMap API.

    Args:
        city_name (str): Name of the city (e.g., "London").

    Returns:
        dict: Weather data including temperature, weather condition, etc.
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': location,
        'appid': "157b97604b50157e6df56194df6f4d94",
        'units': 'metric'  # use 'imperial' for Fahrenheit
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return {'error': f"Failed to get weather data: {response.status_code}"}


In [ ]:
city = "Delhi"
weather_info = get_weather(city)

if 'error' in weather_info:
    print(weather_info['error'])
else:
    weather_report = {
    "city": weather_info['city'],
    "temperature": f"{weather_info['temperature']}°C",
    "condition": weather_info['description'],
    "humidity": f"{weather_info['humidity']}%",
    "wind_speed": f"{weather_info['wind_speed']} m/s"
    }
    print(weather_report)
    


{'city': 'Delhi', 'temperature': '33.69°C', 'condition': 'clear sky', 'humidity': '32%', 'wind_speed': '3.6 m/s'}


In [19]:
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return JSON with function name and arguments as follows:

{"name": <function-name>,"arguments": <args-dict>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city will be mentioned by user"
}
</tools>
"""

In [23]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]

agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current weather in Delhi?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)
dict_output = json.loads(output)
type(dict_output)

{"name": "get_current_weather","arguments": {"location": "Delhi"}}


dict

In [31]:
result = get_weather(**dict_output["arguments"])
# print(result)
# type(result)
str_op = json.dumps(result)
type(str_op)
print(str_op)

{"city": "Delhi", "temperature": 33.69, "description": "clear sky", "humidity": 32, "wind_speed": 3.6}


In [32]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [33]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current weather in Delhi is a clear sky with a temperature of 33.69°C, humidity of 32%, and a wind speed of 3.6 km/h.'